<a href="https://colab.research.google.com/github/nntu079/Basic-Pytorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [76]:
class CNN(nn.Module):
  def __init__(self,in_channels=1,num_classes=10):
    super(CNN,self).__init__()
    
    #input=1x(28,28) => output=8x(28,28) (follow formula in website)
    self.conv1=nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1));
    
    #input=(28,28) => output=8x(14,14)
    self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    
    #input=8x(14,14) => output=16x(7,7)
    self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1));
    
    self.fc1=nn.Linear(16*7*7,num_classes)

  def forward(self,x):
    #output= 8x(28,28)
    x = F.relu(self.conv1(x))

    #output= 8x(14,14)
    x = self.pool(x)

    #output=16x(14,14)
    x = F.relu(self.conv2(x))

    #output=16x(7,7)
    x = self.pool(x)

    #output=(10)
    x = x.reshape(x.shape[0], -1)

    x = self.fc1(x)

    return x

In [77]:
# Load Data
patch=r'/content/drive/MyDrive/Môn học/Pytorch/mnist';
train_dataset=datasets.MNIST(root=patch,train=True,transform=transforms.ToTensor(),download=True)
train_loader= DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root=patch,train=False,transform=transforms.ToTensor(),download=True)
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [78]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [79]:
model= CNN().to(device)
x=torch.randn(64,1,28,28)
print(model(x).shape)

torch.Size([64, 10])


In [80]:
# Hyperparameters
in_channels=1
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=5

In [81]:
# Loss and optimizer
criterion= nn.CrossEntropyLoss();
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [82]:
# Train Network
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):
    # Get data to Cuda if possible
    
    data=data.to(device=device);
    targets=targets.to(device=device);

    # Forward
    scores=model(data);
    loss= criterion(scores,targets);

    # Backward
    optimizer.zero_grad();
    loss.backward();

    # Gradient Descent
    optimizer.step();

In [83]:
# Check accuracy on training and test to see how good our model

def check_accuracy(loader,model):
  num_correct=0;
  num_samples=0;

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)

      scores=model(x);
      _,predictions=scores.max(1);

      num_correct += (predictions==y).sum();
      num_samples += predictions.size(0);

    print(num_correct/num_samples,num_correct,num_samples)

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

tensor(0.9846) tensor(59076) 60000
tensor(0.9845) tensor(9845) 10000
